In [37]:
from astropy.io import fits
from astropy.table import Table
from math import log10
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.misc import imresize
from scipy import misc
from scipy.ndimage import zoom
from scipy.ndimage.interpolation import rotate
import glob

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten

from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.noise import GaussianNoise
from keras.optimizers import rmsprop

import random
import pdb

from sklearn.utils import shuffle

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

from keras.models import model_from_json
from keras.models import model_from_yaml
from keras.optimizers import SGD
from keras.constraints import maxnorm

# Function to read images

In [52]:
def extract_thumb(im,x,y,size):
    if size %2==0:
        size = size+1
    up_x=int(x-size/2)
    dow_x=int(x+size/2)
    up_y=int(y-size/2)
    dow_y=int(y+size/2)
    res=im[up_x:dow_x,up_y:dow_y]        
    return res 

# to read data
def read_data(pathin,pathsave,maxim):
    size_im=69
    size_crop=207
  

    data=fits.getdata(pathin+'Nair_Abraham_cat.fit',1)
    idcat=data['dr7objid']
    ttype=data['TType']
    
    #define the morphologies we want to retrieve
    m=ttype*0-1
    #m[np.where((ttype>=0) & ((ttype<=3)))]=1
    #m[np.where(((ttype<0) & (ttype>=-5)) | ((ttype>3) & (ttype<=10)))]=0
    m[np.where((ttype>=-5) & (ttype<=0))]=0
    m[np.where((ttype>0) & (ttype<=10))]=1
      
    D=np.zeros([maxim,size_im,size_im,3])  #input tensor - images dimensions + color channels
    Y=np.zeros(maxim) #label vector
    idvec=np.zeros([maxim], dtype=np.long)
    
    iteri=-1;
    numim=0;
    numim_init=numim
    nplace=0  #location 1st galaxy to be read 
    catalog=Table(data)

    while iteri<maxim-1:
        try:
            numgal=idcat[numim]
            namegal=str(numgal)+"_GZOO_.jpg"        
            scidata = misc.imread(pathin+'/cutouts_jpeg_all/'+namegal)
            f=numim
            
            print('reading: '+namegal)
            
        except:
            print("Galaxy number %d is missing" % (numim))
            print(namegal)
            numim += 1
            continue
        
        lx,ly, lz=scidata.shape
        #wrong shape - ignore image
        if lx < 256 or ly<256 or m[f]<0:
            numim += 1
            continue

        if lx<size_im:
            numim += 1
            continue

        
        scidata = extract_thumb(scidata,int(lx/2.0),int(ly/2.0),size_crop) # take only a cutout of 207*207 pixels
        scidata=zoom(scidata, [1/3.,1./3,1], order=3)  #keep 1/3 pixels to speed up
        
        
        iteri=iteri+1
        
        #scidata = np.transpose(scidata) #tranpose image
        
        D[iteri,:,:,:]=scidata  #add image to the input tensor
        print(np.max(D[iteri,:,:,:]))

        Y[iteri]=m[f] #update the label
        
        idvec[iteri]=idcat[f] 

        if iteri%5 ==0:
            print("Saving example")
            misc.imsave(pathsave+"examples_stamps/label_"+str(Y[iteri])+"_"+namegal,scidata)
            
         
        numim=numim+1

        
        Y = Y.squeeze()

    # this is to avoid reading all images at every training
    print("Saving image and target vector")
    np.save(pathsave+"image_vector_Sab_"+str(maxim)+".npy",D) 
    np.save(pathsave+"target_vector_Sab_"+str(maxim)+".npy",Y)
    np.save(pathsave+"ID_vector_Sab_"+str(maxim)+".npy",idvec) 

    return D,Y

# Function for model definition

In [70]:
#only two convolutional layers - this can be changed

def CNN_Nair(img_channels, img_rows, img_cols):
    dropoutpar=0.5
    depth=16 #32   
    nb_dense = 64
    # SGD parameters [when using SGD optimizer]
    #lr=0.001   #0.001
    #decay=0
    #momentum=0.9   #0.9
    #nesterov=True
    model = Sequential()
    model.add(Convolution2D(32, 6,6, border_mode='same',
                        input_shape=( img_rows, img_cols,img_channels)))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Convolution2D(64, 5, 5, border_mode='same'))
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Convolution2D(128, 2, 2, border_mode='same'))
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Convolution2D(128, 3, 3, border_mode='same'))
    model.add(Activation('relu'))
    
    model.add(Dropout(0.25))

    #Neuronal Networks starts here
    #-------------------------------#

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(.5)) 
    model.add(Dense(1, init='uniform', activation='sigmoid'))
   
    print("Compilation...")
    
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    print("Model Summary")
    print("===================")
    model.summary()
    return model

# Function to train

In [50]:
def train_convnet_Nair(X,Y,ntrain,nval,test_name):


        
    # train params - hardocded for simplicity
    batch_size = 64 #64
    nb_epoch = 50
    data_augmentation = True
    
    
    ind=random.sample(range(0, ntrain+nval-1), ntrain+nval-1)
    X_train = X[ind[0:ntrain],:,:,:]   
    X_val = X[ind[ntrain:ntrain+nval],:,:,:]
    Y_train = Y[ind[0:ntrain]]
    Y_val = Y[ind[ntrain:ntrain+nval]]

   
    # input image dimensions
    img_rows, img_cols = X_train.shape[1:3]
    img_channels = 3

    
    ### Right shape for X
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols,img_channels)
    X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols,img_channels)


    #Avoid more iterations once convergence
    patience_par=10
    earlystopping = EarlyStopping( monitor='val_loss',patience = patience_par,verbose=0,mode='auto' )
    modelcheckpoint = ModelCheckpoint(test_name+"_best.hd5",monitor='val_loss',verbose=0,save_best_only=True)


    #build model
    model=CNN_Nair(img_channels, img_rows, img_cols)




    if not data_augmentation:
        print('Not using data augmentation.')
        history = model.fit(X_train, Y_train,
                            batch_size=batch_size,
                            nb_epoch=nb_epoch,
                            validation_data=(X_val, Y_val),
                            shuffle=True,
                            verbose=verbose, callbacks=[earlystopping, modelcheckpoint])
    else:
        print('Using real-time data augmentation.')

        # this will do preprocessing and realtime data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False, 
            samplewise_center=False, 
            featurewise_std_normalization=False, 
            samplewise_std_normalization=False,
            zca_whitening=False, 
            rotation_range=45,
            width_shift_range=0.05,  
            height_shift_range=0.05, 
            horizontal_flip=True,
            vertical_flip=True,
            zoom_range=[0.75,1.3])  

        
        #datagen.fit(X_train)
        
        history = model.fit_generator(
                    datagen.flow(X_train, Y_train, batch_size=batch_size),
                    samples_per_epoch=X_train.shape[0],
                    nb_epoch=nb_epoch,
                    validation_data=(X_val, Y_val),
                    callbacks=[ earlystopping, modelcheckpoint]
                )



    print("Saving model...")
    # save weights
    model.save_weights(test_name,overwrite=True)
    
        
    
    return test_name

# Function to test

In [22]:
def test_convnet_Nair(X,model_name):
    
    # input image dimensions
    img_rows, img_cols = X.shape[1:3]
    img_channels = 3
    X = X.reshape(X.shape[0], img_rows, img_cols,img_channels)
    
    #====== load model & predict=======

    print("Loading weights", model_name)
    
    model=CNN_Nair(img_channels, img_rows, img_cols)
    model.load_weights(model_name)
    Y_pred = model.predict_proba(X)


    return Y_pred

# Read

In [68]:
READ_IMAGES=True
LOAD_NPY=False


maxim=3000  #number of images to read in D, Y vectors

pathin="/Users/marchuertascompany/Documents/teaching/big_data_ED/morphology/"
pathsave = "/Users/marchuertascompany/Documents/teaching/big_data_ED/morphology/Sab/"
nparams=1

#file to save model weights
model_name=pathsave+"Nair_Sab.hd5"

## reading

if READ_IMAGES:
    print("Reading images")
    print("====================")
    D,Y=read_data(pathin,pathsave,maxim)  #read images
    
if LOAD_NPY:
    print("Loading D, Y")
    D=np.load(pathsave+"image_vector_TType_"+str(maxim)+".npy")
    Y=np.load(pathsave+"target_vector_TType_"+str(maxim)+".npy")
    ID=np.load(pathsave+"ID_vector_TType_"+str(maxim)+".npy")    

Reading images
reading: 587748927626149924_GZOO_.jpg
243.0
Saving example
reading: 587748927626870899_GZOO_.jpg
255.0
reading: 587748927627722875_GZOO_.jpg


/Users/marchuertascompany/anaconda3/envs/astrophd_tutorial/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/Users/marchuertascompany/anaconda3/envs/astrophd_tutorial/lib/python3.6/site-packages/ipykernel_launcher.py:81: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


222.0
reading: 587722981742084144_GZOO_.jpg
226.0
reading: 587722981744640177_GZOO_.jpg
163.0
reading: 587722981744771128_GZOO_.jpg
190.0
Saving example
reading: 587722981745295552_GZOO_.jpg
202.0
reading: 587722981745426489_GZOO_.jpg
237.0
reading: 587722981747392587_GZOO_.jpg
203.0
reading: 587722981748048006_GZOO_.jpg
192.0
reading: 587722981750276247_GZOO_.jpg
194.0
Saving example
reading: 587722981750931635_GZOO_.jpg
reading: 587722981754011901_GZOO_.jpg
255.0
reading: 587722981755977976_GZOO_.jpg
225.0
reading: 587748928160530448_GZOO_.jpg
255.0
reading: 587748928160858191_GZOO_.jpg
255.0
reading: 587748928162562264_GZOO_.jpg
246.0
Saving example
reading: 587748928166428807_GZOO_.jpg
220.0
reading: 587722982278496424_GZOO_.jpg
158.0
reading: 587722982278758580_GZOO_.jpg
232.0
reading: 587722982278889623_GZOO_.jpg
214.0
reading: 587722982282035411_GZOO_.jpg
232.0
Saving example
reading: 587722982283214886_GZOO_.jpg
234.0
reading: 587722982284460121_GZOO_.jpg
255.0
reading: 5877229

236.0
reading: 587722984440528920_GZOO_.jpg
191.0
reading: 587722984440529128_GZOO_.jpg
246.0
reading: 587722984440659987_GZOO_.jpg
234.0
Saving example
reading: 587722984440856613_GZOO_.jpg
251.0
reading: 587722984442626278_GZOO_.jpg
173.0
reading: 587722984443674817_GZOO_.jpg
254.0
reading: 587722984444854544_GZOO_.jpg
186.0
reading: 588848898822439113_GZOO_.jpg
235.0
Saving example
reading: 588848898824274111_GZOO_.jpg
235.0
reading: 588848898825650236_GZOO_.jpg
255.0
reading: 588848898827288872_GZOO_.jpg
226.0
reading: 588848898827354232_GZOO_.jpg
246.0
reading: 588848898827813018_GZOO_.jpg
251.0
Saving example
reading: 588848898827944016_GZOO_.jpg
200.0
reading: 588848898828468408_GZOO_.jpg
204.0
reading: 588848898828533784_GZOO_.jpg
217.0
reading: 588848898830237807_GZOO_.jpg
252.0
reading: 588848898830237966_GZOO_.jpg
243.0
Saving example
reading: 588848898830565410_GZOO_.jpg
255.0
reading: 588848898832662660_GZOO_.jpg
205.0
reading: 588848898833121335_GZOO_.jpg
255.0
reading: 5

237.0
reading: 588848900439867499_GZOO_.jpg
240.0
reading: 588848900440457365_GZOO_.jpg
245.0
reading: 588848900441768194_GZOO_.jpg
217.0
reading: 588848900441833500_GZOO_.jpg
252.0
Saving example
reading: 588848900441833546_GZOO_.jpg
247.0
reading: 588848900441833621_GZOO_.jpg
255.0
reading: 588848900441899030_GZOO_.jpg
253.0
reading: 588848900441899149_GZOO_.jpg
226.0
reading: 588848900441899191_GZOO_.jpg
249.0
Saving example
reading: 588848900442030164_GZOO_.jpg
248.0
reading: 588848900443668580_GZOO_.jpg
253.0
reading: 588848900443930718_GZOO_.jpg
255.0
reading: 588848900443930723_GZOO_.jpg
210.0
reading: 588848900445241621_GZOO_.jpg
234.0
Saving example
reading: 588848900445765844_GZOO_.jpg
224.0
reading: 588848900446027787_GZOO_.jpg
244.0
reading: 588848900449566953_GZOO_.jpg
205.0
reading: 588848900449829017_GZOO_.jpg
233.0
reading: 588848900450418780_GZOO_.jpg
254.0
Saving example
reading: 588848900451270730_GZOO_.jpg
254.0
reading: 588848900452384975_GZOO_.jpg
238.0
reading: 5

...

220.0
Saving example
reading: 588009372301262882_GZOO_.jpg
246.0
reading: 588009372301328541_GZOO_.jpg
206.0
reading: 588009372835184850_GZOO_.jpg
169.0
reading: 588009372835250255_GZOO_.jpg
255.0
reading: 588009372836102175_GZOO_.jpg
255.0
Saving example
reading: 588009372836102252_GZOO_.jpg
237.0
reading: 588009372836626453_GZOO_.jpg
252.0
reading: 588009372836757529_GZOO_.jpg
reading: 588009372836823138_GZOO_.jpg
255.0
reading: 588009372838133805_GZOO_.jpg
254.0
reading: 588009372838592619_GZOO_.jpg
187.0
Saving example
reading: 588009372839837793_GZOO_.jpg
251.0
reading: 587727942422102080_GZOO_.jpg
255.0
reading: 587727942422429702_GZOO_.jpg
255.0
reading: 587727942422495248_GZOO_.jpg
254.0
reading: 587727942423150773_GZOO_.jpg
231.0
Saving example
reading: 587727942423216175_GZOO_.jpg
223.0
reading: 587727942423281732_GZOO_.jpg
240.0
reading: 587727942424002780_GZOO_.jpg
240.0
reading: 587727942424068115_GZOO_.jpg
249.0
reading: 587727942952419348_GZOO_.jpg
254.0
Saving example
r

248.0
reading: 588010135727243545_GZOO_.jpg
237.0
reading: 588010135727308823_GZOO_.jpg
245.0
reading: 588010135727308993_GZOO_.jpg
229.0
reading: 588010135728685227_GZOO_.jpg
255.0
Saving example
reading: 588010135729143855_GZOO_.jpg
reading: 588010135729143881_GZOO_.jpg
221.0
reading: 588010135729275096_GZOO_.jpg
229.0
reading: 588010135729340508_GZOO_.jpg
242.0
reading: 588010135729537052_GZOO_.jpg
248.0
reading: 588010135729602699_GZOO_.jpg
249.0
Saving example
reading: 588010135729995868_GZOO_.jpg
244.0
reading: 588010135730126895_GZOO_.jpg
254.0
reading: 588010135730651306_GZOO_.jpg
226.0
reading: 588010135730716683_GZOO_.jpg
255.0
reading: 588010135731372160_GZOO_.jpg
231.0
Saving example
reading: 588010135732224152_GZOO_.jpg
231.0
reading: 588010135732289650_GZOO_.jpg
178.0
reading: 588010136263327817_GZOO_.jpg
245.0
reading: 588010136263327818_GZOO_.jpg
250.0
reading: 588010136264638575_GZOO_.jpg
214.0
Saving example
reading: 588010136265687286_GZOO_.jpg
222.0
reading: 5880101

reading: 587728879794913295_GZOO_.jpg
251.0
Saving example
reading: 587728879795699759_GZOO_.jpg
255.0
reading: 587728879796224108_GZOO_.jpg
255.0
reading: 587728879797338373_GZOO_.jpg
178.0
reading: 587728879798124633_GZOO_.jpg
255.0
reading: 587728879799173389_GZOO_.jpg
186.0
Saving example
reading: 587728879801204792_GZOO_.jpg
237.0
reading: 587728879801270382_GZOO_.jpg
255.0
reading: 587728879801270400_GZOO_.jpg
232.0
reading: 587728879802318916_GZOO_.jpg
254.0
reading: 587728879804088328_GZOO_.jpg
251.0
Saving example
reading: 587728879804874799_GZOO_.jpg
238.0
reading: 587728879804940338_GZOO_.jpg
225.0
reading: 587728879805005990_GZOO_.jpg
198.0
reading: 587728879806185615_GZOO_.jpg
255.0
reading: 587728879806972050_GZOO_.jpg
237.0
Saving example
reading: 587728879807692828_GZOO_.jpg
252.0
reading: 587728879807692836_GZOO_.jpg
255.0
reading: 587728879808348176_GZOO_.jpg
244.0
reading: 587728879808806972_GZOO_.jpg
254.0
reading: 587728879808806998_GZOO_.jpg
253.0
Saving example
r

reading: 588010359079436368_GZOO_.jpg
246.0
Saving example
reading: 588010359079501867_GZOO_.jpg
249.0
reading: 588010359079502034_GZOO_.jpg
191.0
reading: 588010359079829550_GZOO_.jpg
250.0
reading: 588010359080222875_GZOO_.jpg
192.0
reading: 588010359080485006_GZOO_.jpg
251.0
Saving example
reading: 588010359080812624_GZOO_.jpg
255.0
reading: 588010359080812714_GZOO_.jpg
234.0
reading: 588010359349379183_GZOO_.jpg
255.0
reading: 588010359080943817_GZOO_.jpg
174.0
reading: 588010359081468085_GZOO_.jpg
192.0
Saving example
reading: 588010359082123398_GZOO_.jpg
224.0
reading: 588010359082975373_GZOO_.jpg
214.0
reading: 588010359083303123_GZOO_.jpg
232.0
reading: 588010359085793482_GZOO_.jpg
225.0
reading: 588010359086579732_GZOO_.jpg
231.0
Saving example
reading: 588010359088218340_GZOO_.jpg
255.0
reading: 588010359603658909_GZOO_.jpg
241.0
reading: 588010359603659087_GZOO_.jpg
224.0
reading: 588010359603920927_GZOO_.jpg
255.0
reading: 588010359605100710_GZOO_.jpg
238.0
Saving example
r

reading: 587728930807545886_GZOO_.jpg
229.0
reading: 587728930807611469_GZOO_.jpg
255.0
reading: 587728930807808035_GZOO_.jpg
255.0
Saving example
reading: 587728930807808089_GZOO_.jpg
255.0
reading: 587728930808135685_GZOO_.jpg
255.0
reading: 587728930808922243_GZOO_.jpg
252.0
reading: 587728930809512050_GZOO_.jpg
247.0
reading: 587728930809643110_GZOO_.jpg
242.0
Saving example
reading: 587728931332489219_GZOO_.jpg
255.0
reading: 587728931333276044_GZOO_.jpg
221.0
reading: 587728931333799975_GZOO_.jpg
230.0
reading: 587728931334389773_GZOO_.jpg
254.0
Saving image and target vector


# Train

In [71]:
ntrain=int(D.shape[0]*8/10)
nval=int(ntrain/10)   
#normalize
##normalization
mu = np.amax(D,axis=(1,2))
print(mu[0,1])

for i in range(0,mu.shape[0]):
    print(np.max(D[i,:,:,0]))
    print(mu.shape)
    D[i,:,:,0] = D[i,:,:,0]/mu[i,0]
    D[i,:,:,1] = D[i,:,:,1]/mu[i,1]
    D[i,:,:,2] = D[i,:,:,2]/mu[i,2]

#D, Y, = shuffle(D,Y,random_state=0)  #change order so that we do not use always the same objects to train/test


print("Training Model")
print("====================")
model_name=train_convnet_Nair(D,Y,ntrain,nval,model_name)

1.0
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.

1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3

1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3

1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3000, 3)
1.0
(3

/Users/marchuertascompany/anaconda3/envs/astrophd_tutorial/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (6, 6), input_shape=(69, 69, 3..., padding="same")`
  
/Users/marchuertascompany/anaconda3/envs/astrophd_tutorial/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), padding="same")`
/Users/marchuertascompany/anaconda3/envs/astrophd_tutorial/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (2, 2), padding="same")`
/Users/marchuertascompany/anaconda3/envs/astrophd_tutorial/lib/python3.6/site-packages/ipykernel_launcher.py:30: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="same")`
/Users/marchuertascompany/anaconda3/envs/astrophd_tutorial/lib/python3.6/site-packages/ipykernel_launcher.py:41: UserWarning: Update

Compilation...
Model Summary
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_99 (Conv2D)           (None, 69, 69, 32)        3488      
_________________________________________________________________
activation_12 (Activation)   (None, 69, 69, 32)        0         
_________________________________________________________________
dropout_78 (Dropout)         (None, 69, 69, 32)        0         
_________________________________________________________________
conv2d_100 (Conv2D)          (None, 69, 69, 64)        51264     
_________________________________________________________________
activation_13 (Activation)   (None, 69, 69, 64)        0         
_________________________________________________________________
max_pooling2d_89 (MaxPooling (None, 34, 34, 64)        0         
_________________________________________________________________
dropout_79 (Dropout)         (None, 34, 34, 64)

/Users/marchuertascompany/anaconda3/envs/astrophd_tutorial/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/Users/marchuertascompany/anaconda3/envs/astrophd_tutorial/lib/python3.6/site-packages/ipykernel_launcher.py:73: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=(array([[[..., callbacks=[<keras.ca..., steps_per_epoch=37, epochs=50)`


Epoch 1/50
37/37 [==============================] - 193s 5s/step - loss: 6.9883 - acc: 0.5617 - val_loss: 6.4036 - val_acc: 0.5983
Epoch 2/50
37/37 [==============================] - 191s 5s/step - loss: 6.9409 - acc: 0.5646 - val_loss: 6.4036 - val_acc: 0.5983
Epoch 3/50
37/37 [==============================] - 186s 5s/step - loss: 6.9750 - acc: 0.5625 - val_loss: 6.4036 - val_acc: 0.5983
Epoch 4/50
 3/37 [=>............................] - ETA: 3:03 - loss: 7.4730 - acc: 0.5312

KeyboardInterrupt: 

# Test

In [25]:


npred=D.shape[0]-(ntrain+nval)  #test sample size; 
pred_index=ntrain+nval          #test sample start index ;


print("Validating model")
print("====================")
Y_pred=test_convnet_Nair(D[pred_index:pred_index+npred,:,:,:],model_name) 
   

Validating model
Loading weights /Users/marchuertascompany/Documents/teaching/big_data_ED/morphology/Sab/Nair_Sab.hd5
Compilation...
... done!
Model Summary
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 69, 69, 16)        448       
_________________________________________________________________
dropout_9 (Dropout)          (None, 69, 69, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 69, 69, 32)        12832     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 34, 34, 32)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 36992)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 64)            

/Users/marchuertascompany/anaconda3/envs/astrophd_tutorial/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu", input_shape=(69, 69, 3..., padding="same", kernel_initializer="orthogonal")`
  
/Users/marchuertascompany/anaconda3/envs/astrophd_tutorial/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", padding="same", kernel_initializer="orthogonal")`
  app.launch_new_instance()
/Users/marchuertascompany/anaconda3/envs/astrophd_tutorial/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="softmax", kernel_initializer="uniform")`
